In [2]:
import carla
import numpy as np
import time
import pyproj

# Connect to CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

# Load the world and choose a vehicle
world = client.get_world()
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.find('vehicle.tesla.model3')
spawn_point = world.get_map().get_spawn_points()[75]
vehicle = world.spawn_actor(vehicle_bp, spawn_point)


In [ ]:
# # Initialize pyproj for coordinate transformation (GPS to ENU)
# proj = pyproj.Proj(proj="latlong", datum="WGS84")  # Latitude, Longitude, Altitude
# enu_proj = pyproj.Proj(proj="utm", zone=33, datum="WGS84")  # UTM projection (adjust if necessary)

In [ ]:
 vehicle.destroy()

In [ ]:
# import pyproj
# # Reference GPS point (latitude, longitude, altitude) at the starting location
# ref_lat = 0.0  # Replace with the actual latitude of your reference point
# ref_lon = 0.0  # Replace with the actual longitude of your reference point
# ref_alt = 0.0  # Replace with the actual altitude of your reference point

# # Set up the pyproj transformer (from geographic to ENU)
# geod = pyproj.Geod(ellps='WGS84')
# proj = pyproj.Proj(proj="latlong", datum="WGS84")

In [ ]:
# # Create transformer for coordinate transformation
# transformer = pyproj.Transformer.from_proj(proj, enu_proj)

# # Function to convert GPS (lat, lon, alt) to ENU coordinates
# def gps_to_enu(lat, lon, alt):
#     east, north, up = transformer.transform(lon, lat, alt)
#     return east, north, up


In [3]:
# Add sensors
gps_bp = blueprint_library.find('sensor.other.gnss')
imu_bp = blueprint_library.find('sensor.other.imu')
lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
gps_bp.set_attribute('sensor_tick', '0.1')
gps = world.spawn_actor(gps_bp, carla.Transform(), attach_to=vehicle)
imu = world.spawn_actor(imu_bp, carla.Transform(), attach_to=vehicle)
lidar = world.spawn_actor(lidar_bp, carla.Transform(carla.Location(z=2.0)), attach_to=vehicle)

In [4]:
# Variables to store data
gps_data = None
imu_data = (0, 0, 0)
lidar_data = []

In [5]:
def is_outlier(gps_data, threshold=0.5):
    lat, lon, alt = gps_data
    return abs(lat) > threshold or abs(lon) > threshold or abs(alt) > threshold


In [6]:
# Sensor callbacks
def gps_callback(data):
    global gps_data
    gps_data = (data.latitude, data.longitude, data.altitude)
    if is_outlier(gps_data):
#         print("Outlier detected in GPS data:", gps_data)
        gps_data = None  
#     else:
#         print(f"GPS Data: {gps_data}") 
#     print(f"GPS Data: {gps_data}")  # Print out GPS data for debugging
def imu_callback(data):
    global imu_data
    imu_data = (data.accelerometer.x, data.accelerometer.y, data.accelerometer.z)

def lidar_callback(data):
    global lidar_data
    points = np.frombuffer(data.raw_data, dtype=np.float32).reshape(-1, 4)
    lidar_data = points

In [7]:
# Attach sensors to listen for data
gps.listen(gps_callback)
imu.listen(imu_callback)
lidar.listen(lidar_callback)

In [8]:
# Initial state and covariance matrix (initialize in ENU coordinates)
# Wait for GPS data to initialize the state
while gps_data is None:
    time.sleep(0.1)  # Wait until we get the GPS data

ref_lat, ref_lon, ref_alt = gps_data  # Use the GPS data for reference

# Convert the GPS coordinates (lat, lon, alt) to ENU coordinates
# initial_enu_coords = gps_to_enu(ref_lat, ref_lon, ref_alt)
state = np.array([ref_lat, ref_lon, ref_alt, 0, 0, 0])  # [x, y, z, vx, vy, vz]
P = np.eye(6) * 0.02  # Initial covariance matrix with small uncertainties

# Define process noise and measurement noise (tunable parameters)
Q = np.diag([0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001])  # Process noise covariance
R_gps = np.diag([0.01, 0.01, 0.01])  # GPS measurement noise
R_lidar = np.diag([0.3, 0.3])  # LiDAR measurement noise

In [9]:
# EKF Prediction Step
def predict(state, P, imu_data, dt):
    ax, ay, az = imu_data

    # State transition matrix
    F = np.eye(6)
    F[0, 3] = F[1, 4] = F[2, 5] = dt

    # Control input based on IMU accelerations
    u = np.array([0, 0, 0, ax, ay, az])

    # Predict the state and update the covariance
    state = F @ state + u * dt
    P = F @ P @ F.T + Q  # Update the covariance with process noise

    return state, P

# GPS Update Step
def update_with_gps(state, P, gps_data):
    lat, lon, alt = gps_data
    # Convert GPS data to ENU coordinates
    lat, lon, alt = gps_data
    z = np.array([lat, lon, alt])  # GPS measurement (latitude, longitude, altitude)

    # Measurement matrix
    H = np.array([
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0]
    ])

    # Measurement residual
    y = z - H @ state
    S = H @ P @ H.T + R_gps
    K = P @ H.T @ np.linalg.inv(S)  # Kalman gain

    # Update state and covariance
    state = state + K @ y
    P = (np.eye(6) - K @ H) @ P

    return state, P

# LiDAR Update Step
def update_with_lidar(state, P, lidar_data):
    if len(lidar_data) > 0:
        # Estimate distance from the number of LiDAR points
        distance = len(lidar_data) * 0.01
        z = np.array([
            state[0] + distance * np.cos(state[2]),  # LiDAR x update
            state[1] + distance * np.sin(state[2])   # LiDAR y update
        ])

        # Measurement matrix for LiDAR
        H = np.array([
            [1, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0]
        ])

        # Measurement residual
        y = z - H @ state
        S = H @ P @ H.T + R_lidar
        K = P @ H.T @ np.linalg.inv(S)  # Kalman gain

        # Update state and covariance
        state = state + K @ y
        P = (np.eye(6) - K @ H) @ P

    return state, P
# LiDAR Update Step
# def update_with_lidar(state, P, lidar_data, R_lidar):
#     if len(lidar_data) > 0:
#         distance = len(lidar_data) * 0.01  
#         z = np.array([
#             state[0] + distance * np.cos(state[2]),  # LiDAR x update
#             state[1] + distance * np.sin(state[2])   # LiDAR y update
#         ])

#         # Measurement matrix for LiDAR
#         H = np.array([
#             [1, 0, -distance * np.sin(state[2]), 0, 0, 0],  # تعديل لزاوية الاتجاه
#             [0, 1, distance * np.cos(state[2]), 0, 0, 0]
#         ])

#         # Measurement residual
#         y = z - H @ state
#         S = H @ P @ H.T + R_lidar
#         K = P @ H.T @ np.linalg.inv(S)  # Kalman gain

#         # Update state and covariance
#         state = state + K @ y
#         P = (np.eye(6) - K @ H) @ P

#     return state, P

In [ ]:
# Main loop
try:
    vehicle.set_autopilot(True)  # Enable autopilot once outside the loop

    while True:
        loop_start = time.time()  # Start time for each loop iteration
        dt = 0.1  # Time step (100 ms)

        # EKF prediction and updates if data is available
        if gps_data is not None and imu_data is not None and len(lidar_data) > 0:
            state, P = predict(state, P, imu_data, dt)  # Prediction step
            state, P = update_with_gps(state, P, gps_data)  # GPS update
            state, P = update_with_lidar(state, P, lidar_data)  # LiDAR update

        # Print the current estimated state (every 10 seconds for clarity)
        if int(time.time()) % 10 == 0:
            print("Current state:", state)
            print("GPS Data:", gps_data)

        # Sleep to maintain a consistent loop cycle time
        elapsed_time = time.time() - loop_start
        time.sleep(max(0, dt - elapsed_time))  # Ensure stable loop time

except KeyboardInterrupt:
    print("Simulation stopped by user.")
finally:
    # Clean up resources
    vehicle.destroy()
    gps.destroy()
    imu.destroy()
    lidar.destroy()

Current state: [-6.53505867e-01  1.33108109e-02  8.98255305e+00 -5.07949690e-01
 -4.15531231e-02  1.69639806e+01]
GPS Data: (0.0018141045108706066, 0.0009154527152103789, 0.0018465614411979914)
Current state: [-6.04349498e-01  1.28080369e-02  8.98090015e+00 -4.46009441e-01
 -3.95361025e-02  1.69919706e+01]
GPS Data: (0.0018141045108706066, 0.0009154527152103789, 0.0017661475576460361)
Current state: [-5.57091094e-01  1.22779851e-02  8.98181478e+00 -3.88559679e-01
 -2.96543609e-02  1.69983828e+01]
GPS Data: (0.0018141045108706066, 0.0009154527152103789, 0.0014544486766681075)
Current state: [-5.14671638e-01  1.37372358e-02  8.98309930e+00 -3.41446260e-01
 -2.70892293e-02  1.70600857e+01]
GPS Data: (0.0018141045108706066, 0.0009154527152103789, 0.001301670097745955)
Current state: [-4.74048862e-01  1.46931758e-02  8.98939278e+00 -3.01701808e-01
 -8.86919620e-03  1.69652136e+01]
GPS Data: (0.0018141045108706066, 0.0009154527152103789, 0.0014514732174575329)
Current state: [-4.35468023e-01

Current state: [-7.04397326e-02  2.97186075e-02  9.06593180e+00  2.74024566e-03
 -2.41392968e-02  1.71678748e+01]
GPS Data: (0.0018379385961821981, -2.743045285901665e-05, 0.0015609931433573365)
Current state: [-6.38248293e-02  3.00292765e-02  8.95588810e+00  8.32552483e-03
  2.43454392e-02  1.69698031e+01]
GPS Data: (0.0018479125056813928, -0.00042531231682538565, 0.0019058417528867722)
Current state: [-6.39711275e-02  3.29093824e-02  8.95891867e+00  1.16617856e-02
  1.64517548e-02  1.70433349e+01]
GPS Data: (0.0018481122196476463, -0.0004338694840519229, 0.0014277457958087325)
Current state: [-6.41403900e-02  3.47126443e-02  8.96764979e+00  4.26875130e-03
 -1.54112467e-02  1.70468775e+01]
GPS Data: (0.0018482173538814095, -0.00043881552280230845, 0.0014198875287547708)
Current state: [-6.38294228e-02  3.29173475e-02  8.97527012e+00  3.64013134e-03
 -4.76906105e-02  1.70310241e+01]
GPS Data: (0.0018482956220253755, -0.00044307808612156595, 0.0012931250967085361)
Current state: [-6.359

Current state: [-0.49600392  0.045575    9.00530766 -0.30394123  0.01832337 16.99782442]
GPS Data: (0.0012165635133385422, -0.00016046385672882266, 0.0012026404729112983)
Current state: [-7.93818580e-03 -2.88733996e-02  8.97072666e+00  1.15988456e-02
 -9.08983385e-02  1.69596756e+01]
GPS Data: (0.0012078732834481798, 8.526734664817565e-05, 0.0011333274887874722)
Current state: [-1.58206825e-02 -2.69919270e-02  8.97050829e+00  5.88030219e-03
 -7.65863984e-02  1.69478914e+01]
GPS Data: (0.0012078458690467642, 8.932620379042933e-05, 0.0011925315484404564)
Current state: [-2.56006649e-02 -2.28851167e-02  8.96934932e+00  2.62512672e-04
 -8.18293432e-02  1.69903902e+01]
GPS Data: (0.0012078153019814408, 9.399043100525511e-05, 0.001291141496039927)
Current state: [-3.00093836e-02 -2.20036158e-02  8.97192988e+00 -2.87124145e-03
 -8.46390508e-02  1.70027794e+01]
GPS Data: (0.0012077878875800252, 9.834264800196002e-05, 0.0011730385012924671)
Current state: [-3.41469724e-02 -2.14562426e-02  8.975

Current state: [ 9.40536262e-02  5.02738831e-03  9.45904970e+00  6.04079279e-01
 -1.46638529e-02  1.75516602e+01]
GPS Data: None
Current state: [ 9.40536262e-02  5.02738831e-03  9.45904970e+00  6.04079279e-01
 -1.46638529e-02  1.75516602e+01]
GPS Data: None
Current state: [ 9.40536262e-02  5.02738831e-03  9.45904970e+00  6.04079279e-01
 -1.46638529e-02  1.75516602e+01]
GPS Data: None
Current state: [ 9.40536262e-02  5.02738831e-03  9.45904970e+00  6.04079279e-01
 -1.46638529e-02  1.75516602e+01]
GPS Data: None
Current state: [ 9.40536262e-02  5.02738831e-03  9.45904970e+00  6.04079279e-01
 -1.46638529e-02  1.75516602e+01]
GPS Data: None
Current state: [ 9.40536262e-02  5.02738831e-03  9.45904970e+00  6.04079279e-01
 -1.46638529e-02  1.75516602e+01]
GPS Data: None
Current state: [ 9.40536262e-02  5.02738831e-03  9.45904970e+00  6.04079279e-01
 -1.46638529e-02  1.75516602e+01]
GPS Data: None
Current state: [ 9.40536262e-02  5.02738831e-03  9.45904970e+00  6.04079279e-01
 -1.46638529e-02 

Current state: [-0.7322136   0.02531358  9.33669211 -1.21731348  0.02533151 17.02429589]
GPS Data: (0.00123442002434615, 0.00020951064650312186, 0.34102365374565125)
Current state: [-0.73264955  0.02460392  9.3377956  -1.2209193   0.03000609 16.97535348]
GPS Data: (0.00123442002434615, 0.00020951037235905322, 0.3412989675998688)
Current state: [-0.73698223  0.02477028  9.33460187 -1.22456522  0.02675377 16.94815379]
GPS Data: (0.00123442002434615, 0.00020951032095704037, 0.3412623107433319)
Current state: [-0.7361408   0.02425832  9.32959475 -1.22461044  0.02723648 16.94699927]
GPS Data: (0.00123442002434615, 0.00020951040662706182, 0.34104907512664795)
Current state: [-7.36822124e-01  2.40543302e-02  9.32528176e+00 -1.22411463e+00
  1.57969484e-02  1.69310584e+01]
GPS Data: (0.00123442002434615, 0.00020951033809104465, 0.34101709723472595)
Current state: [-7.35184266e-01  2.27525909e-02  9.32031172e+00 -1.22056393e+00
  8.28044286e-03  1.69224477e+01]
GPS Data: (0.00123442002434615, 0

Current state: [-9.65586413e-02  4.61676881e-02  8.98249268e+00  8.10508502e-03
 -4.35860555e-03  1.69982029e+01]
GPS Data: (0.001597586210905888, 0.0001020078428286488, 0.0016694259829819202)
Current state: [-9.53439370e-02  4.55918603e-02  8.98371273e+00  1.22089724e-02
 -4.40448608e-03  1.69994808e+01]
GPS Data: (0.001597586210905888, 0.0001020078428286488, 0.0016302871517837048)
Current state: [-9.63249575e-02  4.61886089e-02  8.98485481e+00  1.67955237e-02
 -3.34966863e-03  1.69852646e+01]
GPS Data: (0.001597586210905888, 0.0001020078428286488, 0.0016843032790347934)
Current state: [-9.40448019e-02  4.55046349e-02  8.98461574e+00  1.70125666e-02
 -3.60240448e-03  1.69987058e+01]
GPS Data: (0.001597586210905888, 0.0001020078428286488, 0.0016592406900599599)
Current state: [-9.21558500e-02  4.49148319e-02  8.98554498e+00  1.30034934e-02
 -4.37014631e-03  1.69988658e+01]
GPS Data: (0.001597586210905888, 0.0001020078428286488, 0.0016588210128247738)
